### Laboratorio Mlops Metricas - Bastian Rojas Pereira

#### Para realizar el laboratorio de las metricas de Mlops usando el dataset de Sklearn con datos de la diabetes

In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, train_test_split, LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import Binarizer

In [2]:
# Cargar el dataset de diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

In [3]:
# Convertir el problema de regresión en uno de clasificación binaria
median_y = np.median(y)
y_binary = (y > median_y).astype(int)

# Definir el modelo
model = LinearRegression()

In [4]:
# Técnicas de Validación Cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)
loo = LeaveOneOut()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# K-Fold Cross-Validation
kf_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')

# Leave-One-Out Cross-Validation
loo_scores = cross_val_score(model, X, y, cv=loo, scoring='neg_mean_squared_error')

# Stratified K-Fold Cross-Validation
# Para esto, necesitamos un clasificador que funcione con datos binarios
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=10000)
skf_scores = cross_val_score(log_reg, X, y_binary, cv=skf, scoring='accuracy')

# Métricas de Evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

# Para las métricas de regresión usamos el modelo de regresión lineal
model.fit(X_train, y_train)
y_pred_reg = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_reg)
mae = mean_absolute_error(y_test, y_pred_reg)

# Imprimir resultados
print(f"K-Fold Cross-Validation MSE: {kf_scores.mean()}")
print(f"Leave-One-Out Cross-Validation MSE: {loo_scores.mean()}")
print(f"Stratified K-Fold Cross-Validation Accuracy: {skf_scores.mean()}")
print("Matriz de Confusión:")
print(conf_matrix)
print(f"Exactitud (Accuracy): {accuracy}")
print(f"Precisión (Precision): {precision}")
print(f"Exhaustividad (Recall): {recall}")
print(f"Puntuación F1 (F1 Score): {f1}")
print(f"AUC-ROC: {auc_roc}")
print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"Error Absoluto Medio (MAE): {mae}")

K-Fold Cross-Validation MSE: -3015.381694287272
Leave-One-Out Cross-Validation MSE: -3001.752846999431
Stratified K-Fold Cross-Validation Accuracy: 0.7397088866189989
Matriz de Confusión:
[[37 12]
 [11 29]]
Exactitud (Accuracy): 0.7415730337078652
Precisión (Precision): 0.7073170731707317
Exhaustividad (Recall): 0.725
Puntuación F1 (F1 Score): 0.7160493827160495
AUC-ROC: 0.7400510204081632
Error Cuadrático Medio (MSE): 0.1606713018919572
Error Absoluto Medio (MAE): 0.33434945729724524


### Resultados

## Ahora daremos a conocer el significado de cada resultado
### K-Fold Cross-Validation MSE
- **Valor:** -3015.381694287272
- **Significado:** El error cuadrático medio (MSE) promedio obtenido mediante validación cruzada K-Fold. Este valor negativo indica el MSE promedio en el conjunto de entrenamiento. El MSE mide la diferencia media al cuadrado entre los valores reales y los valores predichos, y un valor más bajo indica un mejor desempeño del modelo.

### Leave-One-Out Cross-Validation MSE
- **Valor:** -3001.752846999431
- **Significado:** Similar al anterior, este es el MSE promedio utilizando validación Leave-One-Out (LOO). En LOO, se usa cada punto de datos como un conjunto de prueba mientras se entrena el modelo en el resto de los datos.

### Stratified K-Fold Cross-Validation Accuracy
- **Valor:** 0.7397088866189989
- **Significado:** La precisión promedio obtenida mediante validación cruzada K-Fold estratificada. La precisión mide la proporción de predicciones correctas, y un valor de aproximadamente 0.74 indica que el modelo tiene un desempeño razonablemente bueno en la clasificación binaria.

### Matriz de Confusión
- **Valor:**
  ```
  [[37 12]
   [11 29]]
  ```
- **Significado:** 
  - **37:** Verdaderos negativos (casos negativos correctamente predichos).
  - **12:** Falsos positivos (casos negativos incorrectamente predichos como positivos).
  - **11:** Falsos negativos (casos positivos incorrectamente predichos como negativos).
  - **29:** Verdaderos positivos (casos positivos correctamente predichos).

### Exactitud (Accuracy)
- **Valor:** 0.7415730337078652
- **Significado:** La proporción de predicciones correctas sobre el total de predicciones. Un valor de 0.74 indica que el modelo es correcto el 74% del tiempo.

### Precisión (Precision)
- **Valor:** 0.7073170731707317
- **Significado:** La precisión es la proporción de verdaderos positivos sobre el total de positivos predichos (verdaderos positivos + falsos positivos). Un valor de 0.71 indica que cuando el modelo predice positivo, es correcto el 71% del tiempo.

### Exhaustividad (Recall)
- **Valor:** 0.725
- **Significado:** La exhaustividad o recall es la proporción de verdaderos positivos sobre el total de positivos reales (verdaderos positivos + falsos negativos). Un valor de 0.725 indica que el modelo captura el 72.5% de los casos positivos.

### Puntuación F1 (F1 Score)
- **Valor:** 0.7160493827160495
- **Significado:** La puntuación F1 es la media armónica de la precisión y el recall. Un valor de 0.716 indica un buen equilibrio entre precisión y recall.

### AUC-ROC
- **Valor:** 0.7400510204081632
- **Significado:** El área bajo la curva ROC mide la capacidad del modelo para distinguir entre clases. Un valor de 0.74 indica que el modelo tiene una buena capacidad para diferenciar entre clases positivas y negativas.

### Error Cuadrático Medio (MSE)
- **Valor:** 0.1606713018919572
- **Significado:** El MSE mide la media de los errores al cuadrado entre los valores reales y los valores predichos. Un valor menor indica un mejor desempeño en términos de error cuadrático.

### Error Absoluto Medio (MAE)
- **Valor:** 0.33434945729724524
- **Significado:** El MAE mide la media de los errores absolutos entre los valores reales y los valores predichos. Un valor menor indica un mejor desempeño en términos de error absoluto.

#### En conclusion, estos resultados muestran que el modelo de regresión logística utilizado para la clasificación binaria tiene un desempeño razonable, con una precisión de aproximadamente 74% y un buen equilibrio entre precisión y recall. Para la regresión, el MSE y el MAE indican el nivel de error en las predicciones continuas.